In [1]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import random
import math 
import copy
import random
import argparse
import torch.optim as optim
import torch.nn as nn
import modeldefine
import numpy as np
from scipy.optimize import minimize
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
alpha = 0.01

def UCB(A, phi):
    #### ucb term
    phi = phi.view(-1,1)
    try:
        tmp, LU = torch.linalg.solve(phi,A)
    except:
        A = A.detach().numpy()
        phi2 = phi.detach().numpy()
        tmp = torch.Tensor(np.linalg.solve(A, phi2))

    return torch.sqrt(torch.matmul(torch.transpose(phi,1,0), tmp))

def calculate_v(contextinfo_list, A, theta):
    vj_list = []
    feature_list = []
    for i in contextinfo_list:
        feature = model(i.to(device)).cpu()
        first_item =  torch.mm( feature.view(1,-1) , theta)
        second_item = alpha * UCB(A, feature)
        vj_list.append((first_item + second_item).item())
        feature_list.append(feature.detach().numpy())
    return np.array(vj_list), feature_list

def update_A(A, info_subset):
    for i in info_subset:
        i = torch.tensor(i, dtype=torch.float32,device=device)
        feature = model(i.to(device)).view(1,-1).cpu()
        A = A + torch.mm(feature.t(), feature)
    return A

def prob(vj_list):
    sum = np.sum(np.exp(vj_list)) + 1
    return [np.exp(vj_list[i]) / sum for i in range(len(vj_list))]  

def revenue(vj_list, reward_list):
    sum = np.sum(np.exp(vj_list)) + 1
    return np.sum(np.multiply(np.exp(vj_list), reward_list) / sum)

def assort(contextinfo_list, reward_list, vj_list):
    length = len(vj_list)
    # sort the contextinfo_list and vj with descending order of reward_list
    sorted_list = sorted(zip(contextinfo_list, vj_list, reward_list), key=lambda x: x[2], reverse=True)
    contextinfo_list = [x[0] for x in sorted_list]
    vj_list = [x[1] for x in sorted_list]
    reward_list = [x[2] for x in sorted_list]

    # calculate the optimal assortment
    optimal_assort = []
    optimal_reward = 0
    index = 1
    for i in range(length):
        if revenue(vj_list[:index], reward_list[:index]) >= optimal_reward:
            optimal_reward = revenue(vj_list[:index], reward_list[:index])
            index += 1
        else:
            break
    return contextinfo_list[:index]

# this is for the linear purchase model when v = x dot theta
def get_linear_purchase(feature_list):
    true_Vlist = [(TRUE_THETA @ feature_list[i].reshape(-1,1)).item() for  i in range(len(feature_list))]
    prob_list = prob(true_Vlist)

    # sample item according to prob_list
    if random.uniform(0,1) < 1 - np.sum(prob_list):
        return np.array([0 for i in range(len(feature_list))])
    else:
        returnlist = [0 for i in range(len(feature_list))]
        indexchoose = random.choices([i for i in range(len(prob_list))], weights = prob_list)[0]
        returnlist[indexchoose] = 1
        return np.array(returnlist)

In [19]:
lambd = 1
def likelihood(theta, feature_list ,y_list):
    # feature's dimension is len * dimension , theta is 1*dimension
    v_list = np.matmul(feature_list, theta.T).reshape(-1)
    ln_prob = np.log(prob(v_list))
    summation = ln_prob * y_list
    return -1 * np.sum(summation)

def likelihood_derivative(theta, feature_list, y_list):
    v_list = np.matmul(feature_list, theta.T).reshape(-1)
    prob_list = prob(v_list)
    summation = np.matmul(np.array(feature_list).T, (y_list - prob_list))
    return -1 * summation

def likelihood_array(theta, feature_list_list, y_list_list):
    summation =  0.5 * lambd * np.dot(theta, theta)
    for i in range(len(feature_list_list)):
        summation += likelihood(theta, feature_list_list[i], y_list_list[i])
    return summation

def likelihood_derivative_array(theta, feature_list_list, y_list_list):
    summation = 0.5 * lambd * theta
    for i in range(len(feature_list_list)):
        summation += likelihood_derivative(theta, feature_list_list[i], y_list_list[i])
    return summation

In [20]:
class CustomLikelihoodLoss(nn.Module): 
    def __init__(self, theta_list):
        super(CustomLikelihoodLoss, self).__init__()
        self.theta_list = theta_list

    def forward(self, output_list, y_list):
        loss = 0
        index = 0
        for output in output_list:  
            y = torch.tensor(y_list[index]).to(device) 
            theta = torch.tensor(self.theta_list[index], dtype= torch.float32).to(device) 
            v = torch.mm(output, theta.view(-1,1)) 
            prob = torch.exp(v) / (torch.sum(torch.exp(v)) + 1)  
            loss += torch.sum(torch.log(prob) * y)  
            index += 1  
        return -loss 


In [21]:
import modeldefine
import torch.optim as optim
model = modeldefine.Model(5,10,10,5).to(device)
# 10 20 20 20 20 20  5
optimizer = optim.Adam(model.parameters(), lr=0.01)


tensor([ 5, 10, 10, 10, 10, 10, 10], dtype=torch.int32)


In [86]:
loss_function = CustomLikelihoodLoss(theta_list)
epochs = 10

for epoch in range(epochs):
    output_list = [model(torch.tensor(a).to(device)) for a in a_list]
    loss = loss_function(output_list, y_list)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}")
      
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch [1/10], Loss: 1420.2215576171875
Epoch [2/10], Loss: 1420.221435546875
Epoch [3/10], Loss: 1420.221435546875
Epoch [4/10], Loss: 1420.221435546875
Epoch [5/10], Loss: 1420.221435546875
Epoch [6/10], Loss: 1420.221435546875
Epoch [7/10], Loss: 1420.221435546875
Epoch [8/10], Loss: 1420.221435546875
Epoch [9/10], Loss: 1420.2213134765625
Epoch [10/10], Loss: 1420.221435546875


In [22]:
CONTEXT_ARRAY = np.load('linear_data/features.npy') 
REWARD_ARRAY = np.load('linear_data/rewards.npy')
TRUE_THETA = np.load('linear_data/theta.npy')

In [34]:
data_length = len(CONTEXT_ARRAY)

# define the hyperparameters
input_size = 20
hidden_size = 20
output_size = 10
num_layers = 10

beta = 0.1

H = 100

# initialize the parameters

theta = np.random.randn(output_size) / np.sqrt(output_size)

LAMBDA = lambd * torch.eye(output_size, dtype=torch.float32)


ass_list = []
purchase_list = []
theta_list = []
for t in range(0,2000):
    context = CONTEXT_ARRAY[t]
    profit = REWARD_ARRAY[t]

    theta_tensor = torch.tensor(theta.reshape(-1,1), dtype=torch.float32)
    v_array = calculate_v(torch.tensor(context,dtype=torch.float32), LAMBDA, theta_tensor)[0]
    assortment = assort(context, profit.tolist()[0], v_array.tolist() )
    purchase_vector = get_linear_purchase(assortment)

    # add t
    ass_list.append(np.array(assortment))
    purchase_list.append(purchase_vector)
    # update the parameters
    LAMBDA = update_A(LAMBDA, assortment)
    
    # update theta using MLE
    
    initial_guess = theta
    result = minimize(likelihood_array, initial_guess, args=(ass_list, purchase_list), method='SLSQP', 
                  constraints={'type':'eq', 'fun': likelihood_derivative_array, 'args':(ass_list, purchase_list)})
    theta = result.x
    print("best parameter",theta)
    theta_list.append(theta)

    # update the neural networks

    if t % H == 99:
        a_list = ass_list[-1*H:]
        y_list = purchase_list[-1*H:]
    
        loss_function = CustomLikelihoodLoss(theta_list)
        epochs = 10

        for epoch in range(epochs):
            output_list = [model(torch.tensor(a,dtype = torch.float32).to(device)) for a in a_list]
            loss = loss_function(output_list, y_list)
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}")
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        theta_list = []

best parameter [ 0.7337686  -0.0587528  -0.33480531 -0.32392347  0.3448034   0.54668735
  0.01502706  0.35956204  0.2494917  -0.02821612]
best parameter [ 0.36875125  0.08879493 -0.40843417 -0.42843091  0.3961314   0.95306063
  0.09209474  0.34940971  0.77698145  0.4479671 ]
best parameter [ 0.18684232  0.10537664 -0.67787819 -0.25972342  0.36876688  1.12860043
 -0.06166713  0.56662712  0.95418629  0.17006884]
best parameter [ 0.40223307  0.05019812 -0.541951   -0.53035944  0.6889483   1.38353902
  0.25428782  0.19508375  0.83265961  0.0808708 ]
best parameter [ 0.45474568  0.20770258 -0.55754655 -0.4136868   1.03029223  1.4745701
  0.05380471  0.34583598  0.82036875 -0.05199631]
best parameter [ 0.51349044 -0.08040316 -0.78956614 -0.49170324  1.1945492   1.3749307
  0.19450318  0.42296975  0.90039688  0.16982402]
best parameter [ 0.60181709  0.16124831 -0.48370815 -0.62156648  1.16056768  1.56424744
  0.09306393  0.46947041  0.78697402  0.13129698]
best parameter [ 0.75449694  0.20844

In [35]:
theta

array([0.35418951, 0.89455072, 0.69363703, 0.46252262, 0.01579746,
       0.30783078, 0.07444429, 1.0353994 , 0.65572051, 0.77480338])

In [36]:
TRUE_THETA

array([[0.37454012, 0.95071431, 0.73199394, 0.59865848, 0.15601864,
        0.15599452, 0.05808361, 0.86617615, 0.60111501, 0.70807258]])

In [41]:
ass_list[0][0]

array([0.00448243, 0.76994016, 0.81208992, 0.31374422, 0.87745478,
       0.18644208, 0.95848417, 0.55289522, 0.00402194, 0.39305549])

In [42]:
model(torch.tensor(ass_list[0],dtype = torch.float32).to(device))

tensor([[0.0000, 0.0000, 0.9219, 0.0000, 3.7261, 0.0000, 0.0000, 5.7784, 7.4131,
         6.8920],
        [0.0000, 0.0000, 0.9227, 0.0000, 3.7279, 0.0000, 0.0000, 5.7824, 7.3984,
         6.8987],
        [0.0000, 0.0000, 0.9223, 0.0000, 3.7288, 0.0000, 0.0000, 5.7825, 7.3961,
         6.9011],
        [0.0000, 0.0000, 0.9265, 0.0000, 3.7214, 0.0000, 0.0000, 5.7886, 7.3877,
         6.8950]], device='cuda:0', grad_fn=<MulBackward0>)

In [43]:
torch.save(model,'testmodel.pth')

In [45]:
np.save('testtheta.npy',theta)